In [25]:
from gurobipy import GRB
import gurobipy as gb
import pandas as pd
import numpy as np

In [26]:
# Create the optimization model
model = gb.Model("Question 2: Puff")

In [27]:
max_demand_regular = [72, 70, 123, 93, 51, 41]
slope_regular = [15, 12, 13, 8, 20, 42]
max_demand_jelly = [66, 76, 116, 86, 56, 46]
slope_jelly = [14, 13, 11, 8, 24, 28]

In [28]:
p_regular = model.addVars(6, lb=0, vtype=GRB.CONTINUOUS, name="Regular Price")
p_jelly = model.addVars(6, lb=0, vtype=GRB.CONTINUOUS, name="Jelly Price")
model.setObjective(gb.quicksum(((p_regular[i] - (2*0.33))*(max_demand_regular[i] - slope_regular[i]*p_regular[i])) + ((p_jelly[i] - (2*0.46))*(max_demand_jelly[i] - slope_jelly[i]*p_jelly[i])) for i in range(6)), GRB.MAXIMIZE)

In [29]:
# Price Constraint
for n in range(2):  
    model.addConstr(p_regular[n] <= p_regular[n + 1], f"Regular Price Constraint {n}")

for n in range(2):  
    model.addConstr(p_jelly[n] <= p_jelly[n + 1], f"Jelly Price Constraint {n}")

In [30]:
# Price Constraint
for n in range(2,5):  
    model.addConstr(p_regular[n] >= p_regular[n + 1], f"Regular Price Constraint {n}")

for n in range(2,5):  
    model.addConstr(p_jelly[n] >= p_jelly[n + 1], f"Jelly Price Constraint {n}")

In [31]:
model.addConstr(gb.quicksum(max_demand_regular[i] - (slope_regular[i]*p_regular[i]) for i in range(6)) <= (222/2), "Regular Supply Constraint")

model.addConstr(gb.quicksum(max_demand_jelly[i] - slope_jelly[i]*p_jelly[i] for i in range(6)) <= (274/2), "Jelly Supply Constraint")

<gurobi.Constr *Awaiting Model Update*>

In [32]:
for i in range(6):
    model.addConstr(max_demand_regular[i] - (slope_regular[i]*p_regular[i]) >= 0, "Regular Supply Constraint")

for i in range(6):
    model.addConstr(max_demand_jelly[i] - slope_jelly[i]*p_jelly[i] >= 0, "Jelly Supply Constraint")

In [33]:
for n in range(6):  
    model.addConstr(1.05*p_regular[n] <= p_jelly[n], "5% Constraint")

In [34]:
for n in range(6):  
    model.addConstr(p_regular[n] <= (2*2.99), "Real-Life Regular Constraint")

for n in range(6):  
    model.addConstr(p_jelly[n] <= (2*3.49), "Real-Life Regular Constraint")

In [35]:
# Solve our model
model.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: Intel(R) Core(TM) i7-8750H CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 42 rows, 12 columns and 68 nonzeros
Model fingerprint: 0x081ca51e
Model has 12 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 4e+01]
  Objective range  [6e+01, 1e+02]
  QObjective range [2e+01, 8e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+00, 3e+02]
Presolve removed 27 rows and 0 columns
Presolve time: 0.00s
Presolved: 15 rows, 12 columns, 38 nonzeros
Presolved model has 12 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 3.700e+01
 Factor NZ  : 1.200e+02
 Factor Ops : 1.240e+03 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     C

In [36]:
# Value of the objective function
print("Profit: ", round(model.objVal,2))

Profit:  1027.95


In [37]:
# Print the decision variables
print(model.printAttr('X'))


    Variable            X 
-------------------------
Regular Price[0]       4.4898 
Regular Price[1]      5.05238 
Regular Price[2]         5.98 
Regular Price[3]         5.98 
Regular Price[4]      2.22222 
Regular Price[5]      0.97619 
Jelly Price[0]      4.71429 
Jelly Price[1]        5.305 
Jelly Price[2]        6.279 
Jelly Price[3]        6.279 
Jelly Price[4]      2.33333 
Jelly Price[5]      1.28143 
None
